<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Langgraph_Agentic_RAG_Cyber_AI_Copilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cyber AI Copilot for Security and Intelligence Domain**

In [1]:
!pip install -qU langchain langchain-groq langchain-pinecone semantic-router langchain-community langchain-core fastembed duckduckgo-search langgraph crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.7/160.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [10]:
import os
from typing import List, Dict, Any, Optional, TypedDict
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END
from langchain_community.tools import DuckDuckGoSearchResults
from dotenv import load_dotenv

GROQ_API_KEY = "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"
PINECONE_API_KEY = "8e15b925-3b96-497d-b20a-08d308782b83"
PINECONE_ENVIRONMENT = "us-east-1"

In [11]:
llm = ChatGroq(temperature=0, model="llama3-8b-8192", api_key=GROQ_API_KEY)
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5", max_length=512)
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone_index = pc.Index("new-cyber-search")
vector_store = PineconeVectorStore(index=pinecone_index, embedding=embeddings)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [12]:
# Define the state
class AgentState(TypedDict):
    messages: List[Dict[str, str]]
    memory: Optional[Dict[str, Any]]

# Define the tools
search_tool = DuckDuckGoSearchResults()

def vector_search(query: str) -> str:
    results = vector_store.similarity_search(query, k=3)
    return "\n".join([doc.page_content for doc in results])

# Define the nodes
def search_duck(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    result = search_tool.run(query)
    state["messages"].append({"role": "tool", "content": f"DuckDuckGo Search Result: {result}"})
    return state

def search_vector(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    result = vector_search(query)
    state["messages"].append({"role": "tool", "content": f"Vector Search Result: {result}"})
    return state

In [13]:
def generate_response(state: AgentState) -> AgentState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an advanced AI copilot specializing in cybersecurity and intelligence. Your role is to provide accurate, up-to-date information and insights on cyber threats, vulnerabilities, and intelligence matters. Use the following guidelines:

1. Analyze the user's query and the provided search results thoroughly.
2. Prioritize the most relevant and recent information from both DuckDuckGo and vector search results.
3. Provide a comprehensive response that addresses the user's query, incorporating insights from both search sources.
4. If there are conflicting pieces of information, acknowledge them and provide a balanced view.
5. Always consider the ethical implications of the information you're providing, especially in sensitive cybersecurity matters.
6. If asked about specific attack techniques or vulnerabilities, provide information for defensive purposes only.
7. Recommend reliable sources for further reading when appropriate.
8. If you're unsure about any information, clearly state that and suggest ways to verify or obtain more accurate data.

Remember, you're assisting in a professional capacity in the cybersecurity and intelligence domain. Maintain a tone that is informative, analytical, and security-conscious."""),
        ("human", "{input}"),
        ("human", "DuckDuckGo Search Result: {duck_result}"),
        ("human", "Vector Search Result: {vector_result}"),
        ("human", "Based on the above information, provide a comprehensive response to the query."),
    ])

    chain = LLMChain(llm=llm, prompt=prompt)

    response = chain.run(
        input=state["messages"][-1]["content"],
        duck_result=next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "DuckDuckGo" in m["content"]), "No DuckDuckGo results."),
        vector_result=next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "Vector Search" in m["content"]), "No vector search results.")
    )

    state["messages"].append({"role": "assistant", "content": response})
    return state

# Define the graph
workflow = StateGraph(AgentState)

# Define the nodes
workflow.add_node("search_duck", search_duck)
workflow.add_node("search_vector", search_vector)
workflow.add_node("generate_response", generate_response)

# Define the edges
workflow.add_edge("search_duck", "search_vector")
workflow.add_edge("search_vector", "generate_response")
workflow.add_edge("generate_response", END)

# Set the entry point
workflow.set_entry_point("search_duck")

# Compile the graph
graph = workflow.compile()

In [16]:
# Function to run the agent
def run_agent(query: str, memory: Optional[Dict[str, Any]] = None) -> AgentState:
    state = AgentState(messages=[{"role": "human", "content": query}], memory=memory)
    result = graph.invoke(state)
    return result

# Example usage
if __name__ == "__main__":
    query = "What are the latest trends in ransomware attacks in India last 3 Months?"
    result = run_agent(query)
    for message in result["messages"]:
        if message["role"] == "assistant":
            print("AI Copilot:", message["content"])

AI Copilot: Ransomware attacks have seen a significant increase in recent years, with a 13% rise in 2023 compared to 2021. This trend is expected to continue, with ransomware groups becoming more sophisticated and attacks becoming more targeted. The current state of ransomware is characterized by a rise in attacks on critical infrastructure, with 14 of the 16 U.S. critical infrastructure sectors being targeted by ransomware attacks, according to the Cybersecurity and Infrastructure Security Agency (CISA).

Ransomware groups typically extort money from their victims by encrypting their files and demanding a random ransom. However, there is a rising trend of them relying on additional methods to extract revenue, such as initial access brokers who sell access to compromised machines on a particular network or within a particular organization.

The 2024 Thales Data Threat Report reveals that 93% of IT professionals believe security threats are increasing in volume or severity, a significan